In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split

# 1.	Create recommender system using ratings.csv file. 

### Looping for best accuracy

In [2]:
ratings = pd.read_csv('ratings.csv')
#ratings.head(50)
df_without_TimeStamp=ratings.drop(['timestamp'],axis=1)

name=['cosine','msd','pearson','pearson_baseline']
boo=[True,False]
for i in name:
    for j in boo:
        reader=Reader()
        data=Dataset.load_from_df(df_without_TimeStamp[['userId','movieId','rating']],reader)
        [trainset,testset]=train_test_split(data,test_size=0.15,shuffle=True)
        rec=KNNWithMeans(k=284,sim_options={'name':i,'user_based':j})
        rec.fit(trainset)
        test_pres=rec.test(testset)
        RMSE=accuracy.rmse(test_pres)
        print('For Name:',i,'and for User_based:',j,'RMSE is:',RMSE)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9013
For Name: cosine and for User_based: True RMSE is: 0.9013281585822304
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8913
For Name: cosine and for User_based: False RMSE is: 0.8913054426112546
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8847
For Name: msd and for User_based: True RMSE is: 0.8847462207230151
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8812
For Name: msd and for User_based: False RMSE is: 0.8812012339625013
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9005
For Name: pearson and for User_based: True RMSE is: 0.9004755837663354
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8926
For Name: pearson and for User_based: False RMSE is: 0.8926131786028707
Estimating biases using als...
Computing the pearso

# Best Fit

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9037
For Name: cosine and for User_based: True RMSE is: 0.9013

In [13]:
reader=Reader()
data=Dataset.load_from_df(df_without_TimeStamp[['userId','movieId','rating']],reader)
[trainset,testset]=train_test_split(data,test_size=0.15,shuffle=True)
rec=KNNWithMeans(k=284,sim_options={'name':'cosine','user_based':True})
rec.fit(trainset)
test_pres=rec.test(testset)
RMSE=accuracy.rmse(test_pres)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9042


# 2.	Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [14]:
reader=Reader()
data=Dataset.load_from_df(df_without_TimeStamp[['userId','movieId','rating']],reader)
[trainset,testset]=train_test_split(data,test_size=0.15,shuffle=True)
rec=KNNWithMeans(k=284,sim_options={'name':'cosine','user_based':True})
rec.fit(trainset)
print('Please enter the UserID and MovieID, Please use pace eperated values for input')
a=input()
x,y=(int(i) for i in a.split())
pred_rating=rec.predict(x,y)
print('For the userID:',x,'and the MovieID:',y,'Predicted rating is:',round(pred_rating[3],1))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Please enter the UserID and MovieID, Please use pace eperated values for input
1 47
For the userID: 1 and the MovieID: 47 Predicted rating is: 4.8


# 3.	Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions). Take userID as a keyboard input. Fix the threshold rating as 2.5

In [17]:
ratings = pd.read_csv('ratings.csv')
df_without_TimeStamp=ratings.drop(['timestamp'],axis=1)
reader=Reader()
data=Dataset.load_from_df(df_without_TimeStamp[['userId','movieId','rating']],reader)
[trainset,testset]=train_test_split(data,test_size=0.2,shuffle=True)
rec=KNNWithMeans(k=284,sim_options={'name':'cosine','user_based':False})
rec.fit(trainset)

print('Please enter your desired UserID or the same UserID as of Step 3:')
userid=int(input())
ls_mov_user=[]
mov__not_user=[]

re_mov_not=ratings[ratings['userId']!=userid]['movieId']
re_mov_not.reset_index(drop=True,inplace=True)
for i in range(0,len(re_mov_not)):
    mov__not_user.append(re_mov_not[i])

df=pd.DataFrame()
df['userId']=[]
df['movieId']=[]
df['rating']=[]
userId_rec=[]
movieId_rec=[]
rating_rec=[]
for i in mov__not_user:
    pred_rating=rec.predict(userid,i)
    userId_rec.append(userid)
    movieId_rec.append(i)
    rating_rec.append(round(pred_rating[3],1))
df['userId']=userId_rec
df['movieId']=movieId_rec
df['rating']=rating_rec
df=df[df['rating']>2.5]
df=df.sort_values('rating',ascending=False)
df.reset_index(drop=True,inplace=True)
df.head(10)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Please enter your desired UserID or the same UserID as of Step 3:
1


,userId,movieId,rating
0,1,318,5.0
1,1,1242,5.0
2,1,2571,5.0
3,1,527,5.0
4,1,318,5.0
5,1,2959,5.0
6,1,48516,5.0
7,1,3468,5.0
8,1,4973,5.0
9,1,3972,5.0


# 4.	Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame.

In [25]:
ratings = pd.read_csv('ratings.csv')
df_without_TimeStamp=ratings.drop(['timestamp'],axis=1)
reader=Reader()
data=Dataset.load_from_df(df_without_TimeStamp[['userId','movieId','rating']],reader)
[trainset,testset]=train_test_split(data,test_size=0.2,shuffle=True)
rec=KNNWithMeans(k=284,sim_options={'name':'cosine','user_based':False})
rec.fit(trainset)

print('Please enter your desired UserID or the same UserID as of Step 3:')
userid=int(input())
ls_mov_user=[]
mov__not_user=[]

re_mov_not=ratings[ratings['userId']!=userid]['movieId']
re_mov_not.reset_index(drop=True,inplace=True)
for i in range(0,len(re_mov_not)):
    mov__not_user.append(re_mov_not[i])

df=pd.DataFrame()
df['userId']=[]
df['movieId']=[]
df['rating']=[]
userId_rec=[]
movieId_rec=[]
rating_rec=[]
for i in mov__not_user:
    pred_rating=rec.predict(userid,i)
    userId_rec.append(userid)
    movieId_rec.append(i)
    rating_rec.append(round(pred_rating[3],1))
df['userId']=userId_rec
df['movieId']=movieId_rec
df['rating']=rating_rec
df=df[df['rating']>2.5]
df=df.sort_values('rating',ascending=False)
df.reset_index(drop=True,inplace=True)
#df.head(10)
dff=df.head(10)
dff.drop(['userId','rating'],axis=1,inplace=True)
sr=dff['movieId']
ls=[]
for i in range(0,len(sr)):
    ls.append(sr[i])

link=pd.read_csv('links.csv')
movie=pd.read_csv('movies.csv')
tag=pd.read_csv('tags.csv')
ratings = pd.read_csv('ratings.csv')
tag.drop(['timestamp'],axis=1,inplace=True)

mov_id=[]
name=[]
imdb_Id=[]
Avg_rating_user_based=[]
Avg_rating_movie_based=[]
genre=[]
tagl=[]

for i in range(0,len(ls)):
    #if len(link_movie_tag[link_movie_tag['movieId']== ls[i]])>=1 :
    lp=[]
    lg=[]
    ln=[]
    lt=[]
    mov_id.append(ls[i])

    sn=movie[movie['movieId'] == ls[i]]['title']
    sn.reset_index(drop=True,inplace=True)
    ln=[sn]
    if len(ln)>=1:
        name.append(ln[0][0])
    else:
        name.append('NaN')
    su=link[link['movieId'] == ls[i]]['imdbId']
    su.reset_index(drop=True,inplace=True)
    lp=[su]
    if len(lp)>=1:
        imdb_Id.append(lp[0][0])
    else:
        imdb_Id.append('NaN')
        
    Avg_rating_user_based.append(round(ratings[ratings['userId']==userid].groupby(['userId']).mean()['rating'].iloc[0],1))
    Avg_rating_movie_based.append(round(ratings[ratings['movieId']==ls[i]].groupby(['movieId']).mean()['rating'].iloc[0],1))
    sg=movie[movie['movieId'] == ls[i]]['genres']
    sg.reset_index(drop=True,inplace=True)
    lg=[sg]
    if len(lg)>=1:
        genre.append(lg[0][0])
    else:
        genre.append('NaN')
    st=tag[tag['movieId'] == ls[i]]['tag']
    st.reset_index(drop=True,inplace=True)
    lt=[st]
    if len(lt[0])>=1:
        tagl.append(lt[0][0])
    else:
        tagl.append('NaN')

final_df=pd.DataFrame()
final_df['mov_id']=mov_id
final_df['name']=name
final_df['imdb_Id']=imdb_Id
final_df['Avg_rating_movie_based']=Avg_rating_movie_based
final_df['Avg_rating_user_based']=Avg_rating_user_based
final_df['genre']=genre
final_df['tag']=tagl
final_df


Computing the cosine similarity matrix...
Done computing similarity matrix.
Please enter your desired UserID or the same UserID as of Step 3:
1


C:\Users\kotad\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,mov_id,name,imdb_Id,Avg_rating_movie_based,Avg_rating_user_based,genre,tag
0,318,"Shawshank Redemption, The (1994)",111161,4.4,4.4,Crime|Drama,prison
1,2068,Fanny and Alexander (Fanny och Alexander) (1982),83922,2.8,4.4,Drama|Fantasy|Mystery,NaN
2,48516,"Departed, The (2006)",407887,4.3,4.4,Crime|Drama|Thriller,way too long
3,2571,"Matrix, The (1999)",133093,4.2,4.4,Action|Sci-Fi|Thriller,martial arts
4,2028,Saving Private Ryan (1998),120815,4.1,4.4,Action|Drama|War,World War II
5,7981,Infernal Affairs (Mou gaan dou) (2002),338564,4.3,4.4,Crime|Drama|Thriller,police
6,2571,"Matrix, The (1999)",133093,4.2,4.4,Action|Sci-Fi|Thriller,martial arts
7,5690,Grave of the Fireflies (Hotaru no haka) (1988),95327,4.2,4.4,Animation|Drama|War,made me cry
8,2020,Dangerous Liaisons (1988),94947,4.0,4.4,Drama|Romance,adultery
9,7615,Desert Hearts (1985),89015,4.5,4.4,Drama,NaN
